In [1]:
n0 = 0
n1 = n0 + 2000

In [2]:
import pandas as pd
import random
import numpy as np

In [3]:
authorship = pd.read_csv('work_authorship.csv')

In [4]:
authorship.head(3)

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,last_pub_year,career_len,scientific_age
0,W1015054358,first,A271637905,I200719446,2008,1999,2022,23,9
1,W1015054358,middle,A2099306368,I204308271,2008,2005,2021,16,3
2,W1015054358,middle,A2116498896,I200719446,2008,2004,2021,17,4


In [5]:
len(authorship)

17419535

In [6]:
inst_completeness =  pd.read_csv('inst_completeness.csv')

In [7]:
inst_completeness.head(3)

,work_id,ratio_inst.
0,W1000000589,0.0
1,W1000004903,1.0
2,W1000011982,1.0


In [8]:
works = list(set(inst_completeness.work_id))

In [9]:
len(works)

3380196

In [10]:
works_inst_not0 = list(set(inst_completeness.work_id[inst_completeness['ratio_inst.']>0]))

In [11]:
len(works_inst_not0)

2512874

In [12]:
authorship = authorship[authorship.work_id.isin(works_inst_not0)]

In [13]:
authorship.head(5)

,work_id,author_position,author_id,institution_id,publication_year,first_pub_year,last_pub_year,career_len,scientific_age
0,W1015054358,first,A271637905,I200719446,2008,1999,2022,23,9
1,W1015054358,middle,A2099306368,I204308271,2008,2005,2021,16,3
2,W1015054358,middle,A2116498896,I200719446,2008,2004,2021,17,4
3,W1015054358,middle,A2154523226,I161318765,2008,2008,2008,0,0
4,W1015054358,middle,A2122755604,I161318765,2008,1968,2021,53,40


In [14]:
target_author = pd.read_csv('target_author.csv')

In [15]:
target_author.head(3)

,author_id,MAGAuthorID,AFTMentorID,FirstMenteeYear
0,A367921,367921,2488,2000
1,A1199224,1199224,95661,2009
2,A1495678,1495678,51349,1963


In [16]:
authors = list(target_author.author_id)
FirstMenteeYear = list(target_author.FirstMenteeYear)

In [17]:
print(len(authors))

25727


#### Case Study

In [18]:
def group_info(author_i,df,backward_window,scientific_age_window):
    auship = df
    
    # Method1
    #mentor's all publications
    # works_i = auship['work_id'][auship.author_id==author_i]
    # print('len(works_i) : ',len(works_i))
    
    # Method2
    #mentor's all last-author publications
    works_i = auship['work_id'][(auship.author_id==author_i)&(auship.author_position=='last')]
    

    auship_i = auship[auship.work_id.isin(works_i)].sort_values(by='publication_year')
    # print('len(auship_i) : ',len(auship_i))

    auship_null = auship_i[~auship_i.institution_id.isnull()]

    # print('the ratio of with institution : ','{:.0%}'.format(len(auship_null)/len(auship_i)))
    
    years_i = auship_i.publication_year
    try:
        start_year = min(years_i)
        end_year = max(years_i)
    except:
        start_year = 2021
        end_year = 2022
    # print('start_year : ',start_year)
    # print('end_year : ',end_year)
    
    Group = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution','current_scientific_age'])
   
    for current_year in range(start_year,end_year+1):
        # print('current_year : ',current_year)
        auship_i_back = auship_i[(auship_i.publication_year>=current_year-backward_window)&
                                (auship_i.publication_year<=current_year)]
        auship_i_back_notnull = auship_i_back[~auship_i_back.institution_id.isnull()]
        
        
        auship_i_back =  auship_i_back.reset_index().drop(columns=['index'])
        #new consideration : current_scientific_age
        csa = np.array(auship_i_back['first_pub_year'] - current_year)*-1
        auship_i_back.loc[:,'current_scientific_age']  = csa
        
        
        if len(auship_i_back)==0:
            ratio_institution = 0
        else:
            ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
            
            # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))
        
        
        # ratio_institution = len(auship_i_back_notnull)/len(auship_i_back)
        # print('the ratio of with institution : ','{:.0%}'.format(ratio_institution))

        if not ratio_institution==0:
            try:
                institution_i = auship_i_back['institution_id'][auship_i_back.author_id==author_i].mode().iloc[0]
                # print('institution_i : ',institution_i)

                group_i = auship_i_back[(auship_i_back.institution_id==institution_i)
                                       &(auship_i_back.current_scientific_age<=scientific_age_window) # see above: current_scientific_age
                                       &(auship_i_back.author_id!=author_i)]

                group_size = len(set(group_i.author_id))
            # print('group_size : ',group_size)
            except:
                group_size = 0
            if group_size>0:
                group_i = group_i.assign(group_size = group_size)
                group_i = group_i.assign(mentor_id = author_i)
                group_i = group_i.assign(current_year = current_year)
                group_i = group_i.assign(ratio_institution = ratio_institution)
            else:
                group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution','current_scientific_age'])
        else:
            group_i = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution','current_scientific_age'])
        Group = pd.concat((Group,group_i))
    return Group

In [19]:
backward_window = 3
scientific_age_window = 10
Data = pd.DataFrame([],columns=['work_id','author_position', 'author_id', 'institution_id', 'publication_year',
                                               'first_pub_year', 'scientific_age', 'group_size', 'mentor_id', 'current_year', 'ratio_institution'])
# Iterate author
# i = random.sample(range(len(authors)),1)[0]
# i = 9465

# n0 = 0
# n1 = n0 + 2000

for i in range(n0,n1):
# print('i : ',i)
    author_i = authors[i]
    # print('author_i : ',author_i)
    author_year = FirstMenteeYear[i]
    # print('author_year : ',author_year)

    data = group_info(author_i,authorship,backward_window,scientific_age_window)
    Data = pd.concat((Data,data))
    if i%10==0:
        print(i)

Data['current_scientific_age'] = Data['current_year'] - Data['first_pub_year']
Data.to_csv(f'groupsize/author_{n0}_{n1}.csv',index=0)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
